## Movielens分析
#### データセットダウンロード

In [ ]:
import glob
import pandas as pd
import collections
import re

In [ ]:
%%bash
cd ../datasets
rm -r ml-20m*
curl -O 'http://files.grouplens.org/datasets/movielens/ml-20m.zip'
unzip ml-20m.zip

In [ ]:
files = glob.glob('../datasets/ml-20m/*.csv')
keys = [file.split("/")[-1].strip(".csv") for file in files]
# 何故かkeyの中にある's'も消去されてしまう... とりあえずこのままで
print("df_dict: {}".format(keys))
df_dict = {key:value for key, value in zip(keys, (pd.read_csv(f) for f in files))}

In [ ]:
for key, df in df_dict.items():
    print(key)
    print(df)
    print("=" * 100)

## 分析
  * tag, title(year), genre, ratingに分けて分析してみる
  * tag系はrelevanceを使いたいので、tag.csvではなくtag-genomeとscore-genomeを使う

#### tag

In [ ]:
df_tag = pd.merge(df_dict['genome-tag'], df_dict['genome-score'], on='tagId')
df_tag = pd.merge(df_tag, df_dict['movie'].loc[:,['movieId','title']], on='movieId')
df_tag = df_tag.loc[:, ['title', 'tag', 'relevance']]
print(df_tag)

* movieとtagの関係を調べてみる

In [ ]:
print(df_tag[df_tag['title'] == 'Toy Story (1995)'].sort_values(by = 'relevance', ascending = False).reset_index(drop=True))

* 1つのmovieずつに全てのtagとそれに対するrelevanceがあったので、relevanceが一定以上低い物を除外した方が良さそうだった
* どの程度の割合で除外すれば良さそうか調べるため、　におけるrelevanceの分布を調べてみる

In [ ]:
df_rel = pd.cut(df_tag['relevance'], 10).value_counts(normalize=True)
print(df_rel)
df_rel.plot(kind="bar", title='relevance per tag')

* tagの約9.6割がrelevance > 0.5 なのでほとんどのtagはいらなそう
* とりあえず活用する場合tag数の1.5パーセント程(≒17)個あれば良さそう？

#### titleとyearの関係

In [ ]:
def movie_title_clean(title):    
  s = re.search('\(([^)]+)', title)
  year = 9999
  if s:
    title = title[:s.span()[0]].strip()
    year = s.group(1)
    # yearが無かったりおかしいものには9999を入れておく
    if str(year).isdigit() and len(year) == 4:
      year = int(year)
    else:
      year = 9999     
  return title, year

In [ ]:
df_movie = df_dict['movie']
df_movie['title'] = df_movie['title'].str.strip()
title_year = df_movie['title'].map(movie_title_clean)
title = title_year.apply(lambda x: x[0])
year = title_year.apply(lambda x: x[1]).rename('year')
df = pd.concat([title, year], axis=1)
print(df)

In [ ]:
year_count = df['year'].value_counts(sort=False).drop(9999).sort_index()
print(year_count)
year_count.plot(xticks=[1880, 1900, 1920, 1940, 1960, 1980, 2000, 2020], title='number of movies by year')

* 1990年以降の映画の数がかなり多く、古い映画になる程数が少ない

#### movieとgenreの関係

In [ ]:
df_movie['genres_list'] = df_movie['genres'].str.split('|')
genres_list = sum(df_movie['genres_list'], [])
c = collections.Counter(genres_list)
df = pd.DataFrame.from_dict(c, orient='index').reset_index().rename(columns={'index':'genre', 0:'count'}).sort_values(by = 'count')
print(df)
df.plot(kind="bar",x=df.columns[0], title='number of movies by genre')

* genreの方がtagより粒度が大きく、ずっと少ない
* genreの方が使い勝手は良さそう

#### movieとratingの関係

In [ ]:
df_dict['movie']['title'] = title
df_dict['movie']['year'] = year
df_rating = pd.merge(df_dict['rating'], df_dict['movie'], on='movieId')
print(df_rating)

In [ ]:
# 各ユーザーがつけるレーティング平均の割合
rating_user_mean = df_rating[['userId','rating']].groupby('userId').mean()
rating_user_mean.plot(kind='hist', title='number of users by rating')

* ユーザーがつけるレーティング平均のほとんどが2.5以上
* 低い評価をつけることがほとんどない

In [ ]:
# 各movieのレーティング平均の割合
rating_movie_mean = df_rating[['movieId','rating']].groupby('movieId').mean()
rating_movie_mean.plot(kind='hist', title='number of movies by rating')

* 多くの映画の評価は2~4点の間に収まっている

In [ ]:
# 評価が存在する映画と存在しない映画の割合
rating_count = len(df_rating['movieId'].unique())
movie_count = len(df_dict['movie']['movieId'].unique())
has_rating = pd.DataFrame(
    {'Yes': [rating_count], 'No': [movie_count - rating_count]}
).apply(lambda x:x/sum(x),axis=1)
print(has_rating)
has_rating.plot(kind="barh", title='Whether the movie has a rating')

* 殆どのmovieにratingはありそう

In [ ]:
# movie毎のレーティングおよびレートの散布図
movie_rating_num = df_rating['movieId'].value_counts().sort_index()
movie_rating_num = pd.DataFrame(movie_rating_num).reset_index().rename(columns={'index':'movieId', 'movieId':'rating_num'})
rating_movie_mean = df_rating[['movieId','rating']].groupby('movieId').mean()
movie_scatter = pd.merge(movie_rating_num, rating_movie_mean, on='movieId').drop('movieId', axis=1)
print(movie_scatter)
movie_scatter.plot(kind='scatter', x='rating', y='rating_num', alpha=0.5, title='rating and rating_num scatter')

* 高いレーティングのものは評価数も多く、低いレーティングのものは評価数が少ない